# Build Agent
Por sí solos, los modelos de lenguaje no pueden realizar acciones; solo generan texto. Un caso de uso importante para LangChain es la creación de agentes. Los agentes son sistemas que utilizan LLM como motores de razonamiento para determinar qué acciones realizar y las entradas necesarias para realizarlas. Tras ejecutar las acciones, los resultados se pueden realimentar al LLM para determinar si se necesitan más acciones o si se puede finalizar. Esto se suele lograr mediante llamadas a herramientas.

En este tutorial, crearemos un agente que puede interactuar con un motor de búsqueda. Podrás hacerle preguntas a este agente, observar cómo llama a la herramienta de búsqueda y conversar con él.

## Bibliotecas

In [2]:
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

import getpass
import os

## End-to-end agent
The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!
```Python
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

```
================================[1m Human Message [0m=================================

hi im bob! and i live in sf

==================================[1m Ai Message [0m==================================

Hello Bob! Since you didn't ask a specific question, I don't need to use any tools right now. I'm an AI assistant created by Anthropic to be helpful, honest, and harmless. Feel free to ask me anything and I'll do my best to provide a useful response or look up information using my capabilities.

```Python
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
```

================================[1m Human Message [0m=================================

whats the weather where I live?

==================================[1m Ai Message [0m==================================

[{'text': 'To get the current weather for your location in San Francisco, I can use the tavily_search_results_json tool:', 'type': 'text'}, {'id': 'toolu_01AKa2MErG1CU3zRiGsvpBud', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01AKa2MErG1CU3zRiGsvpBud)
 Call ID: toolu_01AKa2MErG1CU3zRiGsvpBud
  Args:
    query: san francisco weather
    
=================================[1m Tool Message [0m=================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739994486, 'localtime': '2025-02-19 11:48'}, 'current': {'last_updated_epoch': 1739994300, 'last_updated': '2025-02-19 11:45', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 195, 'wind_dir': 'SSW', 'pressure_mb': 1023.0, 'pressure_in': 30.2, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 100, 'feelslike_c': 12.7, 'feelslike_f': 54.8, 'windchill_c': 9.1, 'windchill_f': 48.4, 'heatindex_c': 10.2, 'heatindex_f': 50.3, 'dewpoint_c': 9.8, 'dewpoint_f': 49.7, 'vis_km': 4.0, 'vis_miles': 2.0, 'uv': 1.4, 'gust_mph': 8.9, 'gust_kph': 14.4}}"}, {"url": "https://world-weather.info/forecast/usa/san_francisco/february-2025/", "content": "Weather in San Francisco in February 2025 (California) - Detailed Weather Forecast for a Month Weather World Weather in San Francisco Weather in San Francisco in February 2025 San Francisco Weather Forecast for February 2025, is based on previous years' statistical data. +59°+50° +59°+52° +59°+50° +61°+52° +59°+50° +61°+50° +61°+52° +63°+52° +61°+52° +61°+50° +61°+50° +61°+50° +59°+50° +59°+50° +61°+50° +61°+52° +59°+50° +59°+48° +57°+48° +59°+50° +59°+48° +59°+50° +57°+46° +61°+50° +61°+50° +59°+50° +59°+48° +59°+50° Extended weather forecast in San Francisco HourlyWeek10-Day14-Day30-DayYear Weather in large and nearby cities Weather in Washington, D.C.+41° Sacramento+55° Pleasanton+55° Redwood City+55° San Leandro+55° San Mateo+54° San Rafael+52° San Ramon+52° South San Francisco+54° Vallejo+50° Palo Alto+55° Pacifica+55° Berkeley+54° Castro Valley+55° Concord+52° Daly City+54° Noverd+52° Sign Hill+54° world's temperature today day day Temperature units"}]

==================================[1m Ai Message [0m==================================

The search results provide the current weather conditions and forecast for San Francisco. According to the data from WeatherAPI, the current temperature in San Francisco is around 55°F (13°C) with light rain and winds around 6 mph. The extended forecast shows temperatures ranging from the upper 40s to low 60s Fahrenheit over the next few weeks.

So in summary, it's a cool, rainy day currently in San Francisco where you live, Bob. Let me know if you need any other details about the weather there!

## Instalación
```bash
%pip install -U langchain-community langgraph langchain-ollama tavily-python langgraph-checkpoint-sqlite langchain-core
```

# Tavily
Usaremos Tavily (un motor de búsqueda) como herramienta. Para usarlo, necesitará obtener y configurar una clave API:
```bash
export TAVILY_API_KEY="..."
```
o
```bash
set TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

In [1]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

Tavily API key:
 ········


## Definiendo herramientas
Primero necesitamos crear las herramientas que queremos usar. Nuestra herramienta principal será Tavily, un motor de búsqueda. Tenemos una herramienta integrada en LangChain para usar fácilmente Tavily como herramienta.

In [3]:
# Defining the search engine with max results of 2
search = TavilySearchResults(max_results=2)
# invoking the search engine with a question
search_results = search.invoke("what is the weather in SF")
print("\nResultado del motor de búsqueda\n")
# Pinrting the results
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]


Resultado del motor de búsqueda

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1746916438, 'localtime': '2025-05-10 15:33'}, 'current': {'last_updated_epoch': 1746916200, 'last_updated': '2025-05-10 15:30', 'temp_c': 14.1, 'temp_f': 57.4, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 10.7, 'wind_kph': 17.3, 'wind_degree': 259, 'wind_dir': 'W', 'pressure_mb': 1016.0, 'pressure_in': 29.99, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 88, 'cloud': 0, 'feelslike_c': 12.8, 'feelslike_f': 55.0, 'windchill_c': 12.3, 'windchill_f': 54.1, 'heatindex_c': 13.4, 'heatindex_f': 56.1, 'dewpoint_c': 11.5, 'dewpoint_f': 52.6, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 7.1, 'gust_mph': 16.3, 'gust

# Usando Modelos de Lenguaje
A continuación, aprendamos cómo utilizar un modelo de lenguaje para llamar herramientas. En este caso Ollama

In [4]:
# Model
model = ChatOllama(
    # Modelo descargado desde Ollama (puedes cambiar el nombre según el modelo disponible)
    model="llama3.2",
    base_url="http://localhost:11434"  # URL local donde corre el servidor de Ollama
)

Puedes llamar al modelo de lenguaje pasando una lista de mensajes. Por defecto, la respuesta es una cadena de contenido.

In [5]:
# Using the model
response = model.invoke([HumanMessage(content="hi!")])
print(response.content)

How can I assist you today?


Ahora podemos ver cómo habilitar este modelo para realizar llamadas a herramientas. Para ello, usamos .bind_tools para proporcionar al modelo de lenguaje el conocimiento de estas herramientas. Por defecto, la respuesta es una cadena de contenido.

In [7]:
model_with_tools = model.bind_tools(tools)

Ahora podemos llamar al modelo. Primero, llamémoslo con un mensaje normal y veamos cómo responde. Podemos examinar tanto el campo de contenido como el campo `tool_calls`.

In [8]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': ''}, 'id': '5873cb88-f651-44c7-b823-b331e4f6e1c9', 'type': 'tool_call'}]


El resultado puede variar dependiendo del modelo. En este caso el modelo Ollama Llama3.2 1B no tiene la capacidad de decidir si usar una herramienta o no sino que  hace el llamado de la herramienta sin importar el contenido del "prompt".
El resultado esperado utilizando OpenAI sería:

ContentString: Hello!

ToolCalls: []

Ahora, intentemos llamarlo con alguna entrada que esperaría que se llamara una herramienta.

In [9]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'SF weather'}, 'id': 'd28d84ab-aa9a-4177-85ca-d72a781574cb', 'type': 'tool_call'}]


Podemos ver que ya no hay texto, pero sí hay una llamada a la herramienta. Nos pide que llamemos a la herramienta de búsqueda de Tavily.

Esto todavía no llama a la herramienta; simplemente nos lo indica. Para llamarla, debemos crear nuestro agente.

## Creando el agente
Ahora que hemos definido las herramientas y el LLM, podemos crear el agente. Usaremos LangGraph para construirlo. Actualmente, usamos una interfaz de alto nivel, pero la ventaja de LangGraph es que esta interfaz está respaldada por una API de bajo nivel altamente controlable, por si se desea modificar la lógica del agente.

Ahora podemos inicializar el agente con el LLM y las herramientas.

Tenga en cuenta que estamos pasando el modelo, no model_with_tools. Esto se debe a que create_react_agent llamará automáticamente a .bind_tools.

In [10]:
agent_executor = create_react_agent(model, tools)

## Ejecutando el agente
¡Ahora podemos ejecutar el agente con algunas consultas! Tenga en cuenta que, por ahora, todas son consultas sin estado (no recordará las interacciones anteriores). Tenga en cuenta que el agente devolverá el estado final al finalizar la interacción (que incluye todas las entradas; más adelante veremos cómo obtener solo las salidas).

Primero, veamos cómo responde cuando no es necesario llamar a una herramienta:

In [11]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='5152eb1e-63ae-4765-b34f-ed6e88fe7eae'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T22:45:43.2900083Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3070829300, 'load_duration': 331760700, 'prompt_eval_count': 188, 'prompt_eval_duration': 521337800, 'eval_count': 22, 'eval_duration': 2217207500, 'model_name': 'llama3.2'}, id='run-1a1d2285-c4fd-42cb-b5f0-cc818887ca28-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current events'}, 'id': 'f91a86b4-f164-4ca5-ad6c-d7d3fc2d2ae0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 188, 'output_tokens': 22, 'total_tokens': 210}),
 ToolMessage(content='[{"title": "Current Events - The New York Times", "url": "https://www.nytimes.com/spotlight/learning-current-events", "content": "Resources for teaching about current events using New York Times content.", "score"

El resultado varia dependiendo el modelo. El resultado esperado es:

[HumanMessage(content='hi!', id='a820fcc5-9b87-457a-9af0-f21768143ee3'),
 AIMessage(content='Hello!', response_metadata={'id': 'msg_01VbC493X1VEDyusgttiEr1z', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 264, 'output_tokens': 5}}, id='run-0e0ddae8-a85b-4bd6-947c-c36c857a4698-0', usage_metadata={'input_tokens': 264, 'output_tokens': 5, 'total_tokens': 269})]

Ahora probemos con un ejemplo donde debería invocar la herramienta.

In [12]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='06b22756-f046-4fc7-8b16-865f5ef7a778'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T22:57:58.2005927Z', 'done': True, 'done_reason': 'stop', 'total_duration': 52552843800, 'load_duration': 38193758300, 'prompt_eval_count': 193, 'prompt_eval_duration': 12082023200, 'eval_count': 22, 'eval_duration': 2265435700, 'model_name': 'llama3.2'}, id='run-b9933972-b0cf-4268-980e-8672cd468cbc-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': '449e5898-44a8-4ca2-a852-09275a273fc5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 193, 'output_tokens': 22, 'total_tokens': 215}),
 ToolMessage(content='[{"title": "Weather in San Francisco in May 2025 (California)", "url": "https://world-weather.info/forecast/usa/san_francisco/may-2025/", "content": "Weather in San Francisco in May 2025. San

## Streaming Messages
Hemos visto cómo se puede llamar al agente con .invoke para obtener una respuesta final. Si el agente ejecuta varios pasos, esto puede tardar un tiempo. Para mostrar el progreso intermedio, podemos retransmitir los mensajes a medida que ocurren.

In [13]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (741c1313-173b-4230-b7d7-6695d3cfb1ed)
 Call ID: 741c1313-173b-4230-b7d7-6695d3cfb1ed
  Args:
    query: sf weather
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco in May 2025 (California)", "url": "https://world-weather.info/forecast/usa/san_francisco/may-2025/", "content": "Weather in San Francisco in May 2025. San Francisco Weather Forecast for ... Saturday, 10 May. +54°. Day. +68°. Clear sky. Sunday, 11 May. +52°. Day. +63°. Clear", "score": 0.9484276}, {"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 

## Streaming tokens
Además de transmitir mensajes, también es útil transmitir tokens. Podemos hacerlo especificando stream_mode="messages".

:: nota

A continuación, usamos message.text(), que requiere langchain-core >=0.3.37.

In [14]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

Unfortunately, I was unable to find a specific weather forecast for San Francisco (SF) due to the format of the output provided.

However, according to historical data, on May 10th, the high temperature in San Francisco is around 68°F (20°C), with a clear sky and no precipitation expected.|

## Añadiendo memoria
Como se mencionó anteriormente, este agente no tiene estado. Esto significa que no recuerda interacciones previas. Para dotarlo de memoria, necesitamos pasar un puntero de control. Al pasar un puntero de control, también debemos pasar un thread_id al invocar al agente (para que sepa desde qué hilo/conversación reanudar).

In [15]:
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T23:35:31.8423458Z', 'done': True, 'done_reason': 'stop', 'total_duration': 29131279000, 'load_duration': 12927043800, 'prompt_eval_count': 190, 'prompt_eval_duration': 13432922300, 'eval_count': 23, 'eval_duration': 2767526800, 'model_name': 'llama3.2'}, id='run-8739974d-b4be-4d8d-a373-b8b4b287a29b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current events news'}, 'id': '160c0424-94c6-4394-8abf-25766af4bc31', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 23, 'total_tokens': 213})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"title": "TIME | Current & Breaking News | National & World Updates", "url": "https://time.com/", "content": "Breaking news and analysis from TIME.com. Politics, world news, photos, video, tech reviews, health, science and entertainment news.", "score": 0.4023222},

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T23:36:53.9468267Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15435623700, 'load_duration': 34867000, 'prompt_eval_count': 908, 'prompt_eval_duration': 11930262700, 'eval_count': 24, 'eval_duration': 3458008700, 'model_name': 'llama3.2'}, id='run-bd03043d-1f9d-4fd8-9335-7fe7713b863c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is your name'}, 'id': '7d1f4470-bfb5-4a2a-9ac2-098b80b3768f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 908, 'output_tokens': 24, 'total_tokens': 932})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"title": "Your Name - Wikipedia", "url": "https://en.wikipedia.org/wiki/Your_Name", "content": "Your Name[b] (Japanese: 君の名は。, Hepburn: Kimi no Na wa) is a 2016 Japanese animated fantasy film written and directed by Makoto Shinkai, produced by CoMix Wave Films, and dist

In [17]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T23:38:27.8463755Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13590600800, 'load_duration': 44766800, 'prompt_eval_count': 191, 'prompt_eval_duration': 10323444700, 'eval_count': 24, 'eval_duration': 3221864800, 'model_name': 'llama3.2'}, id='run-c02baf5f-4daa-41f7-92e3-fcbab14d3c6d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'what is your name'}, 'id': 'c60eee73-7f8d-4bbc-a590-abe9a2d50bf1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 191, 'output_tokens': 24, 'total_tokens': 215})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"title": "Your Name - Wikipedia", "url": "https://en.wikipedia.org/wiki/Your_Name", "content": "Your Name[b] (Japanese: 君の名は。, Hepburn: Kimi no Na wa) is a 2016 Japanese animated fantasy film written and directed by Makoto Shinkai, produced by CoMix Wave Films, and dist

¡Listo! En esta guía rápida, explicamos cómo crear un agente simple. También mostramos cómo transmitir una respuesta, no solo con los pasos intermedios, sino también con tokens. También añadimos memoria para que puedas conversar con ellos. ¡Los agentes son un tema complejo con mucho que aprender!

Para más información sobre agentes, consulta la documentación de LangGraph. Esta incluye su propio conjunto de conceptos, tutoriales y guías prácticas.